In [1]:
from py4j.java_gateway import JavaGateway

import tabula
import camelot
import pdftotext
import pandas as pd
import numpy as np

In [2]:
PDF_1 = "../../task_description/examples/GFS 5760519.pdf"
PDF_2 = "../../task_description/examples/sysco PO#_077-2706434.pdf"

PDF_1:
![GFS_5760519_p1](gfs_5760519_page_1.png)
![GFS_5760519_p2](gfs_5760519_page_2.png)

PDF_2:
![PO_077-2706434_p1](sysco_po#_077-2706434_page_1.png)
![PO_077-2706434_p2](sysco_po#_077-2706434_page_2.png)

## tabula-py 1.4.1

In [3]:
# tabula-py 1.4.1
df_1 = tabula.read_pdf(PDF_1, stream=True, pages="all")
df_1.head()

,1,1 500G,00066958017026 SPICE SESAME SEED BLK,|1257749 96/04,Unnamed: 4,192,7.0000,.1108
0,NaN,NaN,NaN,| 182050,NaN,NaN,NaN,NaN
1,2.0,12 212ML,01899500212117 PEPPERCORN GRN IN BRINE,|1281180 27/07,NaN,27.0,45.00,0.5867
2,NaN,NaN,NaN,| 103000,NaN,NaN,NaN,NaN
3,3.0,6 2L,00066958145422 PICKLE CORNICHON EXTRA FINE,|1307496 12/03,NaN,36.0,59.64,3.1264
4,NaN,NaN,101308,| 101308,NaN,NaN,NaN,NaN


In [4]:
df_2 = tabula.read_pdf(PDF_2, stream=True, pages="all")
df_2.head()

,1 1,454 GM QUALFST,181725,0116871,SPICE CARDAMOM GRND,1,23.4700,.0000
0,2 1,3.2 KG FABBRI,150359,1438769,CHERRY ITALIAN AMARENA,2,94.05,0.0
1,3 1,340 GR ROYALCM,181845,3667809,SPICE DILL SEEDS WHL,1,6.00,0.0
2,4 1,454 GR THAIN,257021,5024967,SUGAR PALM,8,4.65,0.0
3,5 12,725 ML GLDNBOY,093025,5079203,SAUCE FISH THAILAND,1,79.80,0.0
4,6 1,5 KGROYALCM,181886,5367999,SALT SEA PNK HIMALAYAN FI,2,38.89,0.0


## camelot-py 0.8.2

In [5]:
# camelot-py 0.8.2
tables_1 = camelot.read_pdf(PDF_1,
                            flavor="stream",
                            split_text=True,
                            suppress_stdout=True,
                            pages="all")
print(tables_1[0].df.head(25))

                                                    0
0   PO01A5    TO EMAIL :             VENDOR # :   ...
1                               ORDERS@QUALIFIRST.COM
2   5:12:46                           GORDON FOOD ...
3    PO01A5                           MILTON DIVISION
4   2999 JAMES SNOW PARKWAY                       ...
5   MILTON, ON L9T 5G4                            ...
6   PHONE-  905-864-3700       ------ WAREHOUSE --...
7   VENDOR #:    683600                           ...
8   NAME    :  QUALIFIRST FOODS LTD               ...
9   89 CARLINGVIEW DR.                   **      P...
10  *****************************   ETOBICOKE     ...
11  ETOBICOKE       ON M9W 5E4                    ...
12  PHONE   :   416-244-1177                      ...
13  FAX     :   416-244-1737                      ...
14  LEAD-  4 DAYS  FRT-        /CS      /CW       ...
15  BROKER -                               PHONE- ...
16  SPECIAL P.O. INSTRUCTIONS-                    ...
17                          

In [6]:
tables_2 = camelot.read_pdf(PDF_2,
                            flavor="stream",
                            split_text=True,
                            suppress_stdout=True,
                            pages="all")
print(tables_2[0].df.head(25))

                                                    0
0                                ** PURCHASE ORDER **
1                                  SYSCO PETERBOROUGH
2   ----------------------------------------------...
3                                  SYSCO PETERBOROUGH
4                               SYSCO CENTRAL ONTARIO
5                                     65 ELMDALE ROAD
6                           CAVAN MONAGHAN ON K9J 0G5
7   Phone: 705-748-6701                 Fax: 705-8...
8   ----------------------------------------------...
9   PO Number:   077-2706434                      ...
10  PO Date: 8/06/20                              ...
11                             Delivery Date: 8/11/20
12                              Pick Up Date: 8/10/20
13             Vendor: QUALIFIRST FOODS LTD(14513002)
14                                  89 CARLINGVIEW DR
15                                       ETOBICOKE,ON
16  Phone:  416-244-1177                FAX:   000...
17                          

## PDFLayoutTextStripper 2.2.3

In [7]:
# PDFLayoutTextStripper 2.2.3
# preserves layout
# https://stackoverflow.com/questions/51334387/unable-to-launch-gateway-from-python-in-py4j
# https://stackoverflow.com/questions/42826221/py4j-how-to-launch-the-java-gateway-from-python
# https://www.py4j.org/py4j_java_gateway.html#examples

gg = (JavaGateway
      .launch_gateway(
          classpath="../../pdf_layout_text_stripper/python-gateway.jar",
          die_on_exit=True))

pdf_stripper = gg.jvm.io.github.jonathanlink.PythonGateway()

In [8]:
res_1 = pdf_stripper.strip(PDF_1)
str_1 = res_1["payload"].splitlines()
for i in range(30):
    print(str_1[i])

                                                                                                                                                         
 PO01A5         TO  EMAIL    :                    VENDOR    #  :      683600     P.O.   #     :5760519     NAME         :QUALIFIRST       FOODS    LTD   
                ORDERS@QUALIFIRST.COM                                                                                                                    
                                                                                                                                                         
      5:12:46                                           GORDON    FOOD   SERVICE     CDN                                                                 
       PO01A5                                           MILTON    DIVISION                                                                               
                                                        2999   JAMES    SNOW

In [9]:
res_2 = pdf_stripper.strip(PDF_2)
print(res_2)

{'success': 'false', 'error': 'String index out of range: -1'}


In [10]:
gg.close()

## pdftotext 2.1.5

In [11]:
# pdftotext 2.1.5
# preserves layout
# https://stackoverflow.com/questions/14372706/visual-studio-cant-build-due-to-rc-exe
with open(PDF_1, "rb") as f1:
    pdf_1 = pdftotext.PDF(f1)

txt_1 = ("\n\n".join(pdf_1) # append all pages
         .splitlines())

for i in range(35):
    print(txt_1[i])

PO01A5      TO EMAIL :              VENDOR # :   683600 P.O. #   :5760519 NAME       :QUALIFIRST FOODS LTD
            ORDERS@QUALIFIRST.COM

   5:12:46                             GORDON FOOD SERVICE CDN                                                         DMOWERS
    PO01A5                             MILTON DIVISION
                                       2999 JAMES SNOW PARKWAY                                                PAGE-              1
                                       MILTON, ON L9T 5G4                                                     P.O. DATE- 8/10/2020
                                                       PHONE- 905-864-3700        ------ WAREHOUSE -------    P.O. NUMBER-5760519
VENDOR #:      683600                                                             QUALIFIRST FOODS LTD        BUYER - JORDAN GARCIA EXT
NAME    :    QUALIFIRST FOODS LTD                 *****************************   89 CARLINGVIEW DR           PHONE 905-693-3607
             89 CAR

In [12]:
with open(PDF_2, "rb") as f2:
    pdf_2 = pdftotext.PDF(f2)

txt_2 = ("\n\n".join(pdf_2)
         .splitlines())

for i in range(25):
    print(txt_2[i])

                                                        ** PURCHASE ORDER **
                                                         SYSCO PETERBOROUGH
 -----------------------------------------------------------------------------------------------------------------------------------
 SYSCO PETERBOROUGH
 SYSCO CENTRAL ONTARIO
 65 ELMDALE ROAD
 CAVAN MONAGHAN ON K9J 0G5
 Phone: 705-748-6701                 Fax: 705-876-5758
 -----------------------------------------------------------------------------------------------------------------------------------
 PO Number:   077-2706434                                                          Buyer: DIANNE BENCZE              705-876-5775
 PO Date: 8/06/20                                                                  Terms: .0000 0 N/      Lead: 0 Days
 Delivery Date: 8/11/20
 Pick Up Date: 8/10/20
 Vendor: QUALIFIRST FOODS LTD(14513002)
         89 CARLINGVIEW DR
             ETOBICOKE,ON
 Phone:      416-244-1177                   FAX:  

## camelot vs pdftotext

TABLE HEADER (1 line):
![table_header](gfs_5760519_page_1_header.png)

In [13]:
# camelot accuracy
tables_1[0].df.iloc[19, 0] # 1 line

'____ ____ ____    ________ ____            ____ ___________                 _______ __  __ __    ___    _____     _______    _____\nLINE PACK SIZE___ GTIN/VND_CODE_ __________ITEM_DESCRIPTION______________ | ITM#/VN_IT _WH_TH_  _QTY_ __PRICE___ _FREIGHT_ __ALLOW_'

In [14]:
# pdftotext accuracy
for i in range(21, 30): # 9 lines
    print(i, ":", txt_1[i].strip())

21 : LINE ____
22 : ____      SIZE___ ________
23 : PACK ____             ____ __________ITEM_DESCRIPTION______________
24 : GTIN/VND_CODE_           ____ ___________              |     _______
25 : ITM#/VN_IT
26 : __ _WH_TH_
27 : __ __      ___ __PRICE___
28 : _QTY_  _____     _______ __ALLOW_
29 : _FREIGHT_  _____


In [15]:
%%timeit # camelot execution time

parsed = camelot.read_pdf(PDF_1,
                          flavor="stream",
                          split_text=True,
                          suppress_stdout=True,
                          pages="all")

df_list = []

for df in parsed._tables:
    df_list.append(df.df)

#1 line per row    
doc_df_camelot = pd.concat(df_list, ignore_index=True)

2.37 s ± 8.73 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit # pdftotext execution time

with open(PDF_1, "rb") as f1:
    pdf_1 = pdftotext.PDF(f1)
    
doc = ("\n\n".join(pdf_1)
       .splitlines())

# 1 line per row
doc_df_pdftxt = pd.DataFrame(doc)

doc_df_pdftxt[0].replace("", np.nan, inplace=True)
doc_df_pdftxt.dropna(inplace=True)
doc_df_pdftxt[0] = doc_df_pdftxt[0].str.strip()

24 ms ± 104 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
